In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
from bs4 import BeautifulSoup
import requests
import pandas as pd


for dirname, _, filenames in os.walk("./"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

./Project_2.ipynb
./.ipynb_checkpoints/Project_2-checkpoint.ipynb


# 1. Frame the problem
Using the customer description, Define the problem your trying to solve in your own words (remember this is not technial but must be specific so the customer understands the project

In [5]:
#Scrape a website for housing data

# 2. Get the Data 
Define how you recieved the data (provided, gathered..)

In [2]:
website = 'https://homes.com/chicago-il/sold/'
page = requests.get(website)

soup = BeautifulSoup(page.content, "html.parser")

results = soup.find(id="ResultsContainer")
houses = results.find_all("div", class_="property-search")
address = soup.find(id="address")
print(results.prettify())

NameError: name 'requests' is not defined

# 3. Explore the Data
Gain insights into the data you have from step 2, making sure to identify any bias

# 4.Prepare the Data


Apply any data transformations and explain what and why


# 5. Model the data
Using selected ML models, experment with your choices and describe your findings. Finish by selecting a Model to continue with


# 6. Fine Tune the Model

With the select model descibe the steps taken to acheve the best rusults possiable 


# 7. Present
In a customer faceing Document provide summery of finding and detail approach taken


# 8. Launch the Model System
Define your production run code, This should be self susficent and require only your model pramaters 


In [ ]:
def infrence(prams):
    results = m.run(prams)
    return results 